In [ ]:
'''
grand average plots based on pile_all mat files generated by compare_constructs_GCaMP96uf

this cell generates the pkls for the next cell to analyze. if no new data was added, do not run

'''

from scipy.io import loadmat
import numpy as np
import seaborn as sns
import pandas as pd
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

save_on = 1 # 1 to save data and figures
subplot_titles = ['peak dF/F', 'd prime', 'half-rise time (ms)', 'full rise time (ms)', 'half-decay time (ms)']
ap_indices = [0,1,2,3] # 0: 1AP, 1: 3AP, 2:10AP, 3:160AP
plot_mat = loadmat(r'data/unnormPlots_singleWells_struct.mat')
df_cols = subplot_titles.copy()
df_cols.insert(0,'construct')

all_data = plot_mat['unnormPlots_singleWells_struct'][0]

all_constructs = [c[0] for c in all_data['construct']]

# create dataframe
df = pd.DataFrame(columns=df_cols)
for ap_idx in ap_indices:

    for construct_idx in range(len(all_constructs)):
        n_wells = len(all_data[construct_idx][1][ap_idx])
        construct_name = all_data[construct_idx][0][0]

        print(construct_name + ': ' + str(n_wells) + ' wells')
        dff = all_data[construct_idx][1][ap_idx]
        snr = all_data[construct_idx][2][ap_idx]

        # get kinetics, convert to milliseconds
        halfrise = all_data[construct_idx][3][ap_idx]*1000
        fullrise = all_data[construct_idx][4][ap_idx]*1000
        halfdecay = all_data[construct_idx][5][ap_idx]*1000
        dprime = all_data[construct_idx][6][ap_idx]

        current_construct = [construct_name] * n_wells
        df_construct = pd.DataFrame(data=np.array([current_construct, dff, dprime, halfrise, fullrise, halfdecay]).T, columns=df_cols)
        df = df.append(df_construct)

    # cast data columns as floats
    castdict = {x: 'float' for x in subplot_titles}
    df = df.astype(castdict)

    if save_on:
        print('Saving...', end='')
        df.to_pickle('data/grand-avg-data_{}.pkl'.format(ap_idx))
        print('Done')

In [ ]:
from scipy.io import loadmat
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from utils import abbreviate_construct

save_on = 1
figure_size = (2.5,8)
matplotlib.rcParams['pdf.fonttype'] = 42
plt.rcParams.update({'font.size': 7})

hits_label = ['jGCaMP8f', 'jGCaMP8m', 'jGCaMP8s', 'GCaMP6s', 'jGCaMP7f', 'jGCaMP7s', 'XCaMP-Gf']
# hits_label = ['8f', '8m', '8s', '6s', '7f', '7s', 'XCaMP-Gf']
hits_colors = ['#0000ff',  '#ff0000', '#666666',  '#FFD670', '#00ff00',  '#E5A4CB',  '#0099ff']
sns.set_palette(sns.color_palette(hits_colors))


subplot_titles = ['peak dF/F', 'd prime', 'half-rise time (ms)', 'full rise time (ms)', 'half-decay time (ms)']

# magnify the kinetics of the following hits
magnify_hits_label = ['jGCaMP8f', 'jGCaMP8m', 'jGCaMP8s']
all_idx = [0,1,2,3] # all APs to plot
all_idx_str = ['1AP', '3AP', '10AP', '160AP']

for apidx in all_idx:
    print('============{}============'.format(all_idx_str[apidx]))
    df_loaded = pd.read_pickle('data/grand-avg-data_{}.pkl'.format(apidx))
    
    df_hits = df_loaded[df_loaded.construct.isin(hits_label)]
    df_magnify_hits = df_loaded[df_loaded.construct.isin(magnify_hits_label)]
    # get abbreviated names for plotting (e.g. 8f instead of jgcamp8f)
    # df_loaded['construct_abbrev'] = [abbreviate_construct(n) for n in df_loaded['construct']]
    # df_hits =  df_loaded[df_loaded['construct_abbrev'].isin(hits_label)]# load only hits in hits_label
    # df_magnify_hits = df_loaded[df_loaded['construct_abbrev'].isin(magnify_hits_label)]

    f = plt.figure(figsize=figure_size)
    # ax = sns.violinplot(x="construct", y="half-rise time", data=df_hits, inner=None, scale='width')
    f.suptitle(all_idx_str[apidx])
    # sns.set_theme()
    sns.set_style('whitegrid')


    for subplot_idx, var_to_plot in enumerate(subplot_titles):
        plt.subplot(5,1,subplot_idx+1)
        '''ax = sns.stripplot(x="construct",
                           y=var_to_plot, 
                           data=df_hits, 
                           order = hits_label, 
                           size=5,
                          zorder=1)
        '''
        ax = sns.boxplot(x="construct", 
                 y=var_to_plot, 
                 data=df_hits, 
                 order = hits_label, 
                 showfliers=True,
                 fliersize=2,
                 linewidth=1,
                 width=.5,
                 palette = hits_colors,
                 flierprops = {'marker': 'o',
                              'markeredgecolor':'none',
                              'markerfacecolor':'darkgray'}
                )
        '''
        ax = sns.boxplot(x="construct", 
                         y=var_to_plot, 
                         data=df_hits, 
                         order = hits_label, 
                         showfliers=False, 
                         color='darkgray',
                         linewidth=3,
                         width=.5,
                         zorder=100,
                         boxprops={'facecolor':'None', "zorder":10}
                        )
        '''
        
        if subplot_idx == len(subplot_titles)-1:
            plt.xticks(rotation=45, ha='right')
        else:
            ax.tick_params(labelbottom=False)
            plt.xlabel('')
        # need jG8 insets in all 'time'-containing fields
        '''
        if 'time' in var_to_plot and ('160' not in all_idx_str[apidx]):
            # magnification inset
            ins_ax = inset_axes(ax, width="35%", height="60%", loc=2, borderpad=2)
            ns_ax = sns.stripplot(x="construct",
                                y=var_to_plot, 
                                data=df_magnify_hits, 
                                order = magnify_hits_label, 
                                size=5,
                                zorder=1)
            
            ins_ax = sns.boxplot(ax=ins_ax, x="construct", 
                 y=var_to_plot, 
                 data=df_magnify_hits, 
                 order = magnify_hits_label, 
                 showfliers=True,
                 fliersize=2,
                 linewidth=1,
                 width=.5,
                 palette = hits_colors,
                flierprops = {'marker': 'o',
                              'markeredgecolor':'none',
                              'markerfacecolor':'darkgray'}
                )
            
            ins_ax = sns.boxplot(ax=ins_ax, x="construct", 
                                y=var_to_plot, 
                                data=df_magnify_hits, 
                                order = magnify_hits_label, 
                                showfliers=False, 
                                color='darkgray',
                                linewidth=3,
                                width=.5,
                                zorder=100,
                                boxprops={'facecolor':'None', "zorder":10})
            
            ins_ax.set_xlabel(None)
            ins_ax.set_ylabel(None)
            ins_ax.set_xticklabels('')
        '''
        f.tight_layout()
        if save_on:
            plt.savefig('figs/grand_avg_{}.pdf'.format(all_idx_str[apidx]))
    sns.despine()
    plt.show()
    # compute grand average statistics
    means = (df_loaded.groupby('construct').mean()*100).apply(np.round)/100 # just a way to get a reasonable number of sig figs
    stds = (df_loaded.groupby('construct').std()*1000).apply(np.round)/1000
    
    means_stds = means.astype('str')+u"\u00B1"+stds.astype('str')
    display(means_stds)
    if save_on:
        means_stds.to_csv('outputs/grand-avg-summary/means_stds_{}.csv'.format(all_idx_str[apidx]))


## Statistical comparisons of sensors

In [ ]:
# stats
import scipy.stats as ss
import scikit_posthocs as sp
import pandas as pd

vars_to_compare = ['peak dF/F', 'd prime', 'half-rise time (ms)', 'full rise time (ms)', 'half-decay time (ms)']

for apidx in range(4):
    for var_to_compare in vars_to_compare:
        df_loaded = pd.read_pickle('data/grand-avg-data_{}.pkl'.format(apidx))

        # constructs to use for statistics
        stats_subset = ['jGCaMP8f', 'jGCaMP8m', 'jGCaMP8s', 'GCaMP6s', 'jGCaMP7f', 'jGCaMP7s', 'XCaMP-G', 'XCaMP-Gf', 'XCaMP-Gf0']
        
        # constructs for just jG8 vs XCaMP comparison
        # stats_subset = ['jGCaMP8f', 'jGCaMP8m', 'jGCaMP8s', 'XCaMP-G', 'XCaMP-Gf', 'XCaMP-Gf0']
        
        df_stats_subset = df_loaded[df_loaded['construct'].isin(stats_subset)]

        # list of values to compare
        stats_list = [df_stats_subset[df_stats_subset['construct']==c][var_to_compare].to_list() for c in stats_subset]

        #list_stats_subset = [df]
        print('==========={} ({})==========='.format(var_to_compare, all_idx_str[apidx]))
        [_,p] = ss.kruskal(*stats_list, nan_policy='omit')# ss.kruskal(jg8f, jg8m, jg8s, xcg, xcgf, xcgf0)
        print('Kruskal-Wallis test: p = {}'.format(p))

        # Dunn's post-hoc
        pval_table = sp.posthoc_dunn(df_stats_subset, val_col = var_to_compare, group_col='construct', p_adjust='bonferroni')
        pval_table_subset = pval_table.loc[['jGCaMP8f', 'jGCaMP8m', 'jGCaMP8s']][['XCaMP-G', 'XCaMP-Gf', 'XCaMP-Gf0']]
        display(pval_table_subset)